In [ ]:
import numpy as np

In [ ]:
def load_data(fname):
    data = np.loadtxt(fname, delimiter=',')
    X, Y = data[:, 1:].reshape(-1, 28, 28), data[:, 0]
    return X, Y

In [ ]:
def mean_filter(x):
    return x.reshape(14, 2, 14, 2).transpose(0, 2, 1, 3).mean(axis=(2, 3))


def image_to_mps(x):
    mps = []
    for pixel in x.flatten():
        
        M = np.zeros([1, 2, 1], float)
        M[0, 0, 0] = np.cos(np.pi/2*pixel/255)
        M[0, 1, 0] = np.sin(np.pi/2*pixel/255)
        mps.append(M)

    return mps

In [ ]:
def dsum(A, B):
    d = A.shape[1]
    assert d == B.shape[1]
    
    dsum = np.zeros((A.shape[0] + B.shape[0], d, A.shape[2] + B.shape[2]))
    dsum[:A.shape[0], :, :A.shape[2]] = A
    dsum[A.shape[0]:, :, A.shape[2]:] = B

    return dsum


def row(A, B):
    assert A.shape[0] == B.shape[0] == 1
    d = A.shape[1]
    assert d == B.shape[1]
    
    row = np.zeros((1, d, A.shape[2] + B.shape[2]))
    row[:, :, :A.shape[2]] = A
    row[:, :, A.shape[2]:] = B

    return row


def col(A, B):
    assert A.shape[2] == B.shape[2] == 1
    d = A.shape[1]
    assert d == B.shape[1]
    
    col = np.zeros((A.shape[0] + B.shape[0], d, 1))
    col[:A.shape[0], :, :] = A
    col[A.shape[0]:, :, :] = B

    return col


def add(mps_a, mps_b):
    L = len(mps_a)
    assert len(mps_b) == L
    
    mps = [row(mps_a[0], mps_b[0])] + [dsum(mps_a[i], mps_b[i]) for i in range(1, L - 1)] + [col(mps_a[L - 1], mps_b[L - 1])]

    return mps